<a href="https://colab.research.google.com/github/sepidehnaghshineh/LD/blob/main/Data_Processing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Starting with Data Processing

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
cd ./drive/MyDrive/PhD_Project/

/content/drive/MyDrive/PhD_Project


In [ ]:
!pip install pydicom

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 79.2 MB/s eta 0:00:00


In [ ]:
import os
import pandas as pd
import numpy as np
import cv2
import pydicom
import ast
from sklearn.utils import resample
from tqdm import tqdm
import yaml
import shutil

In [ ]:


# Paths
csv_path = "./finding_annotations_train_val_test_split.csv"
training_folder = os.path.join(output_base_path, "images")
labels_folder = os.path.join(output_base_path, "labels")
os.makedirs(training_folder, exist_ok=True)
os.makedirs(labels_folder, exist_ok=True)

base_dir = "/content/drive/MyDrive/PhD_Project"
dicom_base_path = "./physionet.org/files/vindr-mammo/1.0.0/images"  # Update this to your DICOM images path
yolo_dataset_dir = os.path.join(base_dir, "yolo_dataset")



# Create necessary directories
for split in ['train', 'val', 'test']:
    os.makedirs(os.path.join(yolo_dataset_dir, 'images', split), exist_ok=True)
    os.makedirs(os.path.join(yolo_dataset_dir, 'labels', split), exist_ok=True)

# Load the CSV file
csv_path = os.path.join(base_dir, "finding_annotations_train_val_test_split.csv")
df = pd.read_csv(csv_path)

# Handle multiple findings and bounding boxes
df['finding_categories'] = df['finding_categories'].apply(ast.literal_eval)

# Convert lists to tuples for grouping
df['finding_categories'] = df['finding_categories'].apply(tuple)

# Split the dataset into training, validation, and test sets
df_train = df[df['split'] == 'train']
df_val = df[df['split'] == 'val']
df_test = df[df['split'] == 'test']

# Define the class list explicitly
class_list = [
    'Mass', 'Suspicious Calcification', 'Asymmetry', 'Focal Asymmetry',
    'Global Asymmetry', 'Architectural Distortion', 'Skin Thickening',
    'Skin Retraction', 'Nipple Retraction', 'Suspicious Lymph Node', 'No Finding'
]

# Ensure unique_classes is ordered according to the predefined class list
unique_classes = [cls for cls in class_list if cls in set([category for categories in df['finding_categories'] for category in categories])]

# Aggregate bounding boxes and findings for each image ID
def aggregate_bboxes(df):
    def combine_lists(x):
        # Convert series to list
        values = list(x)
        # If the first item is a list or tuple, flatten it
        if len(values) > 0 and (isinstance(values[0], (list, tuple))):
            return [item for sublist in values for item in sublist]
        return values

    # First convert tuples back to lists for finding_categories
    df = df.copy()
    df['finding_categories'] = df['finding_categories'].apply(lambda x: list(x) if isinstance(x, tuple) else x)

    return df.groupby('image_id').agg({
        'study_id': 'first',
        'finding_categories': combine_lists,
        'xmin': combine_lists,
        'ymin': combine_lists,
        'xmax': combine_lists,
        'ymax': combine_lists
    }).reset_index()

df_train_aggregated = aggregate_bboxes(df_train)
df_val_aggregated = aggregate_bboxes(df_val)
df_test_aggregated = aggregate_bboxes(df_test)

def balance_dataset(df, target_min_samples=500, random_state=42):
    # Separate majority and minority classes
    df_majority = df[df['finding_categories'].apply(lambda x: 'No Finding' in x)]
    df_minority = df[df['finding_categories'].apply(lambda x: 'No Finding' not in x)]

    # Downsample majority class
    df_majority_downsampled = resample(df_majority,
                                     replace=False,
                                     n_samples=target_min_samples,
                                     random_state=random_state)

    print(f"Original majority class shape: {df_majority.shape}")
    print(f"Downsampled majority class shape: {df_majority_downsampled.shape}")

    # Apply oversampling to minority classes
    oversampled_df = pd.DataFrame()
    oversampling_info = {}  # Track oversampling counts

    for class_name in class_list:
        if class_name == 'No Finding':  # Skip No Finding class as it's handled separately
            continue

        class_df = df_minority[df_minority['finding_categories'].apply(lambda x: class_name in x)]
        original_count = len(class_df)

        if original_count == 0:
            print(f"Warning: No samples found for class {class_name}, skipping...")
            continue

        if original_count < target_min_samples:
            # Calculate how many times we need to duplicate each image
            oversample_factor = target_min_samples // original_count + 1

            # Create multiple copies of the class dataframe
            oversampled_class_df = pd.concat([class_df] * oversample_factor)
            # Take only what we need
            oversampled_class_df = oversampled_class_df.head(target_min_samples)

            # Store oversampling information
            for idx, row in class_df.iterrows():
                image_id = row['image_id']
                if image_id not in oversampling_info:
                    oversampling_info[image_id] = oversample_factor - 1  # -1 because we keep original

            oversampled_df = pd.concat([oversampled_df, oversampled_class_df])
            print(f"Oversampled {class_name}: {original_count} → {len(oversampled_class_df)}")
        else:
            oversampled_df = pd.concat([oversampled_df, class_df])
            print(f"Class {class_name} already has sufficient samples: {original_count}")

    # Combine oversampled data with the majority classes
    df_balanced = pd.concat([df_majority_downsampled, oversampled_df])
    df_balanced = df_balanced.reset_index(drop=True)

    print(f"\nFinal balanced dataset shape: {df_balanced.shape}")
    print(f"Number of images to be oversampled: {len(oversampling_info)}")

    return df_balanced, oversampling_info

# Apply balancing to training set
df_train_balanced, oversampling_info = balance_dataset(df_train_aggregated, target_min_samples=500)

# Apply balancing to validation and test sets without oversampling
df_val_balanced = df_val_aggregated  # No oversampling for validation set
df_test_balanced = df_test_aggregated  # No oversampling for test set

# Modify create_yolo_annotations function
def create_yolo_annotations(df, output_image_folder, output_label_folder, oversampling_info=None, is_oversampled=False):
    os.makedirs(output_image_folder, exist_ok=True)
    os.makedirs(output_label_folder, exist_ok=True)

    for _, row in tqdm(df.iterrows(), total=len(df), desc="Processing images"):
        study_id = row['study_id']
        image_id = row['image_id']

        # Load and process original image
        image_path = os.path.join(dicom_base_path, study_id, f"{image_id}.dicom")
        if not os.path.exists(image_path):
            print(f"Warning: Image not found - {image_path}")
            continue

        dicom_data = pydicom.dcmread(image_path)
        image = dicom_data.pixel_array
        image = (image - image.min()) / (image.max() - image.min()) * 255
        image = image.astype(np.uint8)
        image = cv2.cvtColor(image, cv2.COLOR_GRAY2RGB)

        # Prepare YOLO format annotations
        height, width = image.shape[:2]
        bboxes = []

        # Get categories and coordinates
        categories = row['finding_categories']
        xmins = row['xmin']
        ymins = row['ymin']
        xmaxs = row['xmax']
        ymaxs = row['ymax']

        if not isinstance(categories, (list, tuple)):
            categories = [categories]
            xmins = [xmins]
            ymins = [ymins]
            xmaxs = [xmaxs]
            ymaxs = [ymaxs]

        if 'No Finding' not in categories:
            for category, xmin, ymin, xmax, ymax in zip(categories, xmins, ymins, xmaxs, ymaxs):
                if pd.isna(xmin) or pd.isna(ymin) or pd.isna(xmax) or pd.isna(ymax):
                    continue

                x_center = (xmin + xmax) / 2 / width
                y_center = (ymin + ymax) / 2 / height
                bbox_width = (xmax - xmin) / width
                bbox_height = (ymax - ymin) / height

                try:
                    class_id = unique_classes.index(category)
                    bboxes.append(f"{class_id} {x_center} {y_center} {bbox_width} {bbox_height}")
                except ValueError:
                    print(f"Warning: Category {category} not found in unique_classes")
                    continue

        # Save original image and its annotation
        cv2.imwrite(os.path.join(output_image_folder, f"{image_id}.jpg"), image)
        with open(os.path.join(output_label_folder, f"{image_id}.txt"), 'w') as f:
            f.write("\n".join(bboxes))

        # If this image needs oversampling, create the duplicates
        if is_oversampled and oversampling_info and image_id in oversampling_info:
            num_copies = oversampling_info[image_id]
            for i in range(num_copies):
                # Save duplicate image with unique identifier
                duplicate_id = f"{image_id}_copy_{i+1}"
                cv2.imwrite(os.path.join(output_image_folder, f"{duplicate_id}.jpg"), image)

                # Save duplicate annotation
                with open(os.path.join(output_label_folder, f"{duplicate_id}.txt"), 'w') as f:
                    f.write("\n".join(bboxes))

# Create YOLO annotations with oversampling for training set
create_yolo_annotations(
    df_train_balanced,
    os.path.join(yolo_dataset_dir, 'images', 'train'),
    os.path.join(yolo_dataset_dir, 'labels', 'train'),
    oversampling_info=oversampling_info,
    is_oversampled=True
)

# Create YOLO annotations for validation and test sets (without oversampling)
create_yolo_annotations(
    df_val_balanced,
    os.path.join(yolo_dataset_dir, 'images', 'val'),
    os.path.join(yolo_dataset_dir, 'labels', 'val')
)
create_yolo_annotations(
    df_test_balanced,
    os.path.join(yolo_dataset_dir, 'images', 'test'),
    os.path.join(yolo_dataset_dir, 'labels', 'test')
)

# Create data.yaml file
data_yaml_path = os.path.join(yolo_dataset_dir, "data.yaml")
with open(data_yaml_path, "w") as f:
    f.write(f"train: {os.path.join(yolo_dataset_dir, 'images', 'train')}\n")
    f.write(f"val: {os.path.join(yolo_dataset_dir, 'images', 'val')}\n")
    f.write(f"test: {os.path.join(yolo_dataset_dir, 'images', 'test')}\n")
    f.write(f"nc: {len(unique_classes)}\n")
    f.write(f"names: {unique_classes}\n")

# Function to count images per class
def count_images_per_class(df, unique_classes):
    class_counts = {cls: 0 for cls in unique_classes}
    for _, row in df.iterrows():
        for category in row['finding_categories']:
            class_counts[category] += 1
    return class_counts

# Count images per class for each split
train_class_counts = count_images_per_class(df_train_balanced, unique_classes)
val_class_counts = count_images_per_class(df_val_balanced, unique_classes)
test_class_counts = count_images_per_class(df_test_balanced, unique_classes)

# Create a DataFrame to store the counts
report_df = pd.DataFrame({
    'Class': unique_classes,
    'Train': [train_class_counts[cls] for cls in unique_classes],
    'Validation': [val_class_counts[cls] for cls in unique_classes],
    'Test': [test_class_counts[cls] for cls in unique_classes]
})

# Save the report to a CSV file
report_csv_path = os.path.join(yolo_dataset_dir, "class_distribution_report.csv")
report_df.to_csv(report_csv_path, index=False)

print(f"Class distribution report saved to {report_csv_path}")

# Verify the processed validation data
print("\nVerifying validation dataset:")
val_img_dir = os.path.join(yolo_dataset_dir, 'images', 'val')
val_label_dir = os.path.join(yolo_dataset_dir, 'labels', 'val')

print(f"Validation images: {len(os.listdir(val_img_dir))}")
print(f"Validation labels: {len(os.listdir(val_label_dir))}")

# Check a sample image and its label
if os.listdir(val_img_dir):
    sample_image = os.listdir(val_img_dir)[0]
    sample_label = sample_image.replace('.jpg', '.txt')

    print(f"\nSample validation image: {sample_image}")
    if os.path.exists(os.path.join(val_label_dir, sample_label)):
        print("Label file exists")
        with open(os.path.join(val_label_dir, sample_label), 'r') as f:
            print("Label contents:")
            print(f.read())





Original majority class shape: (11064, 7)
Downsampled majority class shape: (500, 7)
Class Mass already has sufficient samples: 670
Oversampled Suspicious Calcification: 243 → 500
Oversampled Asymmetry: 58 → 500
Oversampled Focal Asymmetry: 158 → 500
Oversampled Global Asymmetry: 16 → 500
Oversampled Architectural Distortion: 76 → 500
Oversampled Skin Thickening: 34 → 500
Oversampled Skin Retraction: 12 → 500
Oversampled Nipple Retraction: 26 → 500
Oversampled Suspicious Lymph Node: 30 → 500

Final balanced dataset shape: (5670, 7)
Number of images to be oversampled: 546


Processing images: 100%|██████████| 5670/5670 [1:30:05<00:00,  1.05it/s]
Processing images: 0it [00:00, ?it/s]
Processing images: 100%|██████████| 4000/4000 [1:34:41<00:00,  1.42s/it]


Class distribution report saved to /content/drive/MyDrive/PhD_Project/yolo_dataset/class_distribution_report.csv

Verifying validation dataset:
Validation images: 0
Validation labels: 0


In [ ]:
import os
import pandas as pd
import numpy as np
import cv2
import pydicom
import ast
from sklearn.utils import resample
from tqdm import tqdm
import yaml
import shutil

# Define paths
csv_path = "./finding_annotations_train_val_test_split.csv"
base_dir = "/content/drive/MyDrive/PhD_Project"
dicom_base_path = "./physionet.org/files/vindr-mammo/1.0.0/images"
yolo_dataset_dir = os.path.join(base_dir, "yolo_dataset")

# Create base directories
training_folder = os.path.join(yolo_dataset_dir, "images")
labels_folder = os.path.join(yolo_dataset_dir, "labels")
os.makedirs(training_folder, exist_ok=True)
os.makedirs(labels_folder, exist_ok=True)

# Create split-specific directories
for split in ['train', 'val', 'test']:
    os.makedirs(os.path.join(yolo_dataset_dir, 'images', split), exist_ok=True)
    os.makedirs(os.path.join(yolo_dataset_dir, 'labels', split), exist_ok=True)

# Load the CSV file
df = pd.read_csv(csv_path)
print("\nCSV Loading Debug:")
print(f"CSV file exists: {os.path.exists(csv_path)}")
print(f"Total rows in CSV: {len(df)}")
print("\nSplit distribution in CSV:")
print(df['split'].value_counts())

# Handle multiple findings and bounding boxes
df['finding_categories'] = df['finding_categories'].apply(ast.literal_eval)

# Convert lists to tuples for grouping
df['finding_categories'] = df['finding_categories'].apply(tuple)

# Split the dataset - only get validation data
df_val = df[df['split'] == 'validation']
print(f"\nNumber of validation entries: {len(df_val)}")

# Define the class list explicitly
class_list = [
    'Mass', 'Suspicious Calcification', 'Asymmetry', 'Focal Asymmetry',
    'Global Asymmetry', 'Architectural Distortion', 'Skin Thickening',
    'Skin Retraction', 'Nipple Retraction', 'Suspicious Lymph Node', 'No Finding'
]

# Ensure unique_classes is ordered according to the predefined class list
unique_classes = [cls for cls in class_list if cls in set([category for categories in df['finding_categories'] for category in categories])]

# Aggregate bounding boxes and findings for each image ID
def aggregate_bboxes(df):
    def combine_lists(x):
        # Convert series to list
        values = list(x)
        # If the first item is a list or tuple, flatten it
        if len(values) > 0 and (isinstance(values[0], (list, tuple))):
            return [item for sublist in values for item in sublist]
        return values

    # First convert tuples back to lists for finding_categories
    df = df.copy()
    df['finding_categories'] = df['finding_categories'].apply(lambda x: list(x) if isinstance(x, tuple) else x)

    return df.groupby('image_id').agg({
        'study_id': 'first',
        'finding_categories': combine_lists,
        'xmin': combine_lists,
        'ymin': combine_lists,
        'xmax': combine_lists,
        'ymax': combine_lists
    }).reset_index()

# Define create_yolo_annotations function
def create_yolo_annotations(df, output_image_folder, output_label_folder, oversampling_info=None, is_oversampled=False):
    print(f"\nProcessing dataset for folder: {output_image_folder}")
    print(f"Number of images to process: {len(df)}")

    os.makedirs(output_image_folder, exist_ok=True)
    os.makedirs(output_label_folder, exist_ok=True)

    for _, row in tqdm(df.iterrows(), total=len(df), desc="Processing images"):
        study_id = row['study_id']
        image_id = row['image_id']

        # Load and process original image
        image_path = os.path.join(dicom_base_path, study_id, f"{image_id}.dicom")
        if not os.path.exists(image_path):
            print(f"Warning: Image not found - {image_path}")
            continue

        dicom_data = pydicom.dcmread(image_path)
        image = dicom_data.pixel_array
        image = (image - image.min()) / (image.max() - image.min()) * 255
        image = image.astype(np.uint8)
        image = cv2.cvtColor(image, cv2.COLOR_GRAY2RGB)

        # Prepare YOLO format annotations
        height, width = image.shape[:2]
        bboxes = []

        # Get categories and coordinates
        categories = row['finding_categories']
        xmins = row['xmin']
        ymins = row['ymin']
        xmaxs = row['xmax']
        ymaxs = row['ymax']

        if not isinstance(categories, (list, tuple)):
            categories = [categories]
            xmins = [xmins]
            ymins = [ymins]
            xmaxs = [xmaxs]
            ymaxs = [ymaxs]

        if 'No Finding' not in categories:
            for category, xmin, ymin, xmax, ymax in zip(categories, xmins, ymins, xmaxs, ymaxs):
                if pd.isna(xmin) or pd.isna(ymin) or pd.isna(xmax) or pd.isna(ymax):
                    continue

                x_center = (xmin + xmax) / 2 / width
                y_center = (ymin + ymax) / 2 / height
                bbox_width = (xmax - xmin) / width
                bbox_height = (ymax - ymin) / height

                try:
                    class_id = unique_classes.index(category)
                    bboxes.append(f"{class_id} {x_center} {y_center} {bbox_width} {bbox_height}")
                except ValueError:
                    print(f"Warning: Category {category} not found in unique_classes")
                    continue

        # Save original image and its annotation
        output_image_path = os.path.join(output_image_folder, f"{image_id}.jpg")
        output_label_path = os.path.join(output_label_folder, f"{image_id}.txt")

        cv2.imwrite(output_image_path, image)
        with open(output_label_path, 'w') as f:
            f.write("\n".join(bboxes))

        print(f"Saved: {output_image_path} and {output_label_path}")

        # If this image needs oversampling, create the duplicates
        if is_oversampled and oversampling_info and image_id in oversampling_info:
            num_copies = oversampling_info[image_id]
            for i in range(num_copies):
                # Save duplicate image with unique identifier
                duplicate_id = f"{image_id}_copy_{i+1}"
                cv2.imwrite(os.path.join(output_image_folder, f"{duplicate_id}.jpg"), image)

                # Save duplicate annotation
                with open(os.path.join(output_label_folder, f"{duplicate_id}.txt"), 'w') as f:
                    f.write("\n".join(bboxes))

# Aggregate bounding boxes for validation set only
df_val_aggregated = aggregate_bboxes(df_val)

# Create YOLO annotations for validation set only
create_yolo_annotations(
    df_val_aggregated,
    os.path.join(yolo_dataset_dir, 'images', 'val'),
    os.path.join(yolo_dataset_dir, 'labels', 'val')
)

# Verify the processed validation data
print("\nVerifying validation dataset:")
val_img_dir = os.path.join(yolo_dataset_dir, 'images', 'val')
val_label_dir = os.path.join(yolo_dataset_dir, 'labels', 'val')

print(f"Validation images directory: {val_img_dir}")
print(f"Validation labels directory: {val_label_dir}")
print(f"Validation images: {len(os.listdir(val_img_dir)) if os.path.exists(val_img_dir) else 0}")
print(f"Validation labels: {len(os.listdir(val_label_dir)) if os.path.exists(val_label_dir) else 0}")

# Check a sample image and its label
if os.path.exists(val_img_dir) and os.listdir(val_img_dir):
    sample_image = os.listdir(val_img_dir)[0]
    sample_label = sample_image.replace('.jpg', '.txt')

    print(f"\nSample validation image: {sample_image}")
    if os.path.exists(os.path.join(val_label_dir, sample_label)):
        print("Label file exists")
        with open(os.path.join(val_label_dir, sample_label), 'r') as f:
            print("Label contents:")
            print(f.read())

# Create data.yaml file
data_yaml_path = os.path.join(yolo_dataset_dir, "data.yaml")
with open(data_yaml_path, "w") as f:
    f.write(f"train: {os.path.join(yolo_dataset_dir, 'images', 'train')}\n")
    f.write(f"val: {os.path.join(yolo_dataset_dir, 'images', 'val')}\n")
    f.write(f"test: {os.path.join(yolo_dataset_dir, 'images', 'test')}\n")
    f.write(f"nc: {len(unique_classes)}\n")
    f.write(f"names: {unique_classes}\n")

# Function to count images per class
def count_images_per_class(df, unique_classes):
    class_counts = {cls: 0 for cls in unique_classes}
    for _, row in df.iterrows():
        for category in row['finding_categories']:
            class_counts[category] += 1
    return class_counts

# Count images per class for each split
train_class_counts = count_images_per_class(df_train_balanced, unique_classes)
val_class_counts = count_images_per_class(df_val_balanced, unique_classes)
test_class_counts = count_images_per_class(df_test_balanced, unique_classes)

# Create a DataFrame to store the counts
report_df = pd.DataFrame({
    'Class': unique_classes,
    'Train': [train_class_counts[cls] for cls in unique_classes],
    'Validation': [val_class_counts[cls] for cls in unique_classes],
    'Test': [test_class_counts[cls] for cls in unique_classes]
})

# Save the report to a CSV file
report_csv_path = os.path.join(yolo_dataset_dir, "class_distribution_report.csv")
report_df.to_csv(report_csv_path, index=False)

print(f"Class distribution report saved to {report_csv_path}")

# Load and check CSV
df = pd.read_csv(csv_path)
print("\nCSV Split Distribution:")
print(df['split'].value_counts())

# Check validation data in CSV
df_val = df[df['split'] == 'validation']
print(f"\nNumber of validation entries in CSV: {len(df_val)}")
print("\nSample validation entries:")
print(df_val[['study_id', 'image_id', 'finding_categories']].head())

# Check actual files
val_img_dir = os.path.join(yolo_dataset_dir, 'images', 'val')
val_label_dir = os.path.join(yolo_dataset_dir, 'labels', 'val')

print("\nChecking actual files:")
print(f"Validation images directory exists: {os.path.exists(val_img_dir)}")
print(f"Validation labels directory exists: {os.path.exists(val_label_dir)}")

if os.path.exists(val_img_dir):
    val_images = [f for f in os.listdir(val_img_dir) if f.endswith(('.jpg', '.jpeg', '.png', '.dicom'))]
    print(f"Number of validation images found: {len(val_images)}")
    if val_images:
        print("Sample validation images:", val_images[:5])

if os.path.exists(val_label_dir):
    val_labels = [f for f in os.listdir(val_label_dir) if f.endswith('.txt')]
    print(f"Number of validation labels found: {len(val_labels)}")
    if val_labels:
        print("Sample validation labels:", val_labels[:5])

# Compare CSV entries with actual files
if os.path.exists(val_img_dir):
    csv_image_ids = set(df_val['image_id'].astype(str) + '.jpg')
    actual_image_ids = set(os.listdir(val_img_dir))

    print("\nComparing CSV entries with actual files:")
    print(f"Images in CSV but not in directory: {len(csv_image_ids - actual_image_ids)}")
    print(f"Images in directory but not in CSV: {len(actual_image_ids - csv_image_ids)}")

    if len(csv_image_ids - actual_image_ids) > 0:
        print("\nSample missing images:")
        print(list(csv_image_ids - actual_image_ids)[:5])


CSV Loading Debug:
CSV file exists: True
Total rows in CSV: 20486

Split distribution in CSV:
split
train         12388
test           4095
validation     4003
Name: count, dtype: int64

Number of validation entries: 4003

Processing dataset for folder: /content/drive/MyDrive/PhD_Project/yolo_dataset/images/val
Number of images to process: 3884


Processing images:   0%|          | 1/3884 [00:24<26:03:26, 24.16s/it]

Saved: /content/drive/MyDrive/PhD_Project/yolo_dataset/images/val/000470cbf12fe2b285cba99286a9a4fa.jpg and /content/drive/MyDrive/PhD_Project/yolo_dataset/labels/val/000470cbf12fe2b285cba99286a9a4fa.txt


Processing images:   0%|          | 2/3884 [00:25<11:38:17, 10.79s/it]

Saved: /content/drive/MyDrive/PhD_Project/yolo_dataset/images/val/000611f8c6a44659a1813f4019241829.jpg and /content/drive/MyDrive/PhD_Project/yolo_dataset/labels/val/000611f8c6a44659a1813f4019241829.txt


...

Saved: /content/drive/MyDrive/PhD_Project/yolo_dataset/images/val/ffeb0fb001fc1be92b7464d557f52579.jpg and /content/drive/MyDrive/PhD_Project/yolo_dataset/labels/val/ffeb0fb001fc1be92b7464d557f52579.txt


Processing images: 100%|██████████| 3884/3884 [1:33:06<00:00,  1.44s/it]

Saved: /content/drive/MyDrive/PhD_Project/yolo_dataset/images/val/fffdc4e37fd29095823d6d3e4d1bf338.jpg and /content/drive/MyDrive/PhD_Project/yolo_dataset/labels/val/fffdc4e37fd29095823d6d3e4d1bf338.txt

Verifying validation dataset:
Validation images directory: /content/drive/MyDrive/PhD_Project/yolo_dataset/images/val
Validation labels directory: /content/drive/MyDrive/PhD_Project/yolo_dataset/labels/val


Validation images: 3884
Validation labels: 3884

Sample validation image: 000470cbf12fe2b285cba99286a9a4fa.jpg
Label file exists
Label contents:



NameError: name 'df_train_balanced' is not defined